In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate
from sklearn import preprocessing

In [2]:
def getDemoPaths(directories):
    demoPaths = []
    for directory in directories:
        for filename in os.listdir(directory):
            demoPaths.append(os.path.join(directory, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    killData, roundData, damageData = [], [], [],
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (killData, roundData, damageData)

In [5]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [6]:
def createSidedCSVDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []

    Kills = []
    Deaths = []
    KdDifferential = []
    DamageDealt = []
    DamageTaken = []
    DamageDifferential = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            
            totalRounds.append(tempTotalRounds)
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [7]:
def createDifferentialDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []

    Kills = []
    Deaths = []
    KdDifferential = []
    DamageDealt = []
    DamageTaken = []
    DamageDifferential = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            
            
            Deaths.append(tempCTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            
            
            totalRounds.append(tempTotalRounds)
            
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [8]:
root_demo_directory = "/mnt/d/Top_Players"

In [9]:
base_output_path =  './damageDifferentials/six_months/'

In [10]:
directories = []
for filename in os.listdir(root_demo_directory):
       directories.append(os.path.join(root_demo_directory, filename))

In [11]:
demos = getDemoPaths(directories)

In [12]:
mirage_demos, inferno_demos = [],[]
for x in demos:
    if "mirage" in x:
        mirage_demos.append(x)
    elif "inferno" in x:
        inferno_demos.append(x)

In [13]:
mirageMappingPath = './mappings/mappings/'
infernoMappingPath = './mappings/mappings/de_inferno_place_mappings.txt'

In [14]:
file = open(infernoMappingPath, 'r')
lines = file.readlines()
file.close()

In [15]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [16]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
            mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [17]:
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[1])

In [18]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [19]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [20]:
mappedPlaceName = {}
for x in lines:
    if(x != ''):
            mappedPlaceName[x.split(':')[0]] = x.split(':')[1]

In [21]:
mirage_demos

['/mnt/d/Top_Players/Blast_Premier_Global_Final_2020/complexity-vs-natus-vincere-m1-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Global_Final_2020/evil-geniuses-vs-furia-m2-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Global_Final_2020/g2-vs-furia-m2-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Global_Final_2020/natus-vincere-vs-g2-m2-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Global_Final_2020/vitality-vs-complexity-m1-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/complexity-vs-evil-geniuses-m2-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/complexity-vs-g2-m1-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/natus-vincere-vs-mibr-m1-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/nip-vs-big-m2-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/vitality-vs-complexity-m1-mirage.dem',
 '/mnt/d/Top_Players/Blast_Premier_Spring_Showdown_2021/furia-vs-mibr-m1-mirage.dem',
 '/mnt/d/Top

In [22]:
mirage_demos.pop(6)

'/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/complexity-vs-g2-m1-mirage.dem'

In [23]:
mirage_demos[6]

'/mnt/d/Top_Players/Blast_Premier_Spring_Groups_2021/natus-vincere-vs-mibr-m1-mirage.dem'

In [24]:
len(inferno_demos)

108

In [25]:
mirage_parsers = generateDemoParsers(mirage_demos)

16:00:49 [INFO] Go version>=1.14.0
16:00:49 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Blast_Premier_Global_Final_2020/complexity-vs-natus-vincere-m1-mirage.dem
16:00:49 [INFO] Setting demo id to complexity-vs-natus-vincere-m1-mirage
16:00:49 [INFO] Setting parse rate to 128
16:00:49 [INFO] Go version>=1.14.0
16:00:49 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Blast_Premier_Global_Final_2020/evil-geniuses-vs-furia-m2-mirage.dem
16:00:49 [INFO] Setting demo id to evil-geniuses-vs-furia-m2-mirage
16:00:49 [INFO] Setting parse rate to 128
16:00:49 [INFO] Go version>=1.14.0
16:00:49 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Blast_Premier_Global_Final_2020/g2-vs-furia-m2-mirage.dem
16:00:49 [INFO] Setting demo id to g2-vs-furia-m2-mirage
16:00:49 [INFO] Setting parse rate to 128
16:00:49 [INFO] Go version>=1.14.0
16:00:49 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Blast_Premier_Global_Final_2020

16:00:51 [INFO] Setting parse rate to 128
16:00:51 [INFO] Go version>=1.14.0
16:00:51 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Dreamhack Masters Spring 2021/spirit-vs-g2-mirage.dem
16:00:51 [INFO] Setting demo id to spirit-vs-g2-mirage
16:00:51 [INFO] Setting parse rate to 128
16:00:51 [INFO] Go version>=1.14.0
16:00:51 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Dreamhack Masters Spring 2021/vitality-vs-big-m1-mirage.dem
16:00:51 [INFO] Setting demo id to vitality-vs-big-m1-mirage
16:00:51 [INFO] Setting parse rate to 128
16:00:51 [INFO] Go version>=1.14.0
16:00:51 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Dreamhack_Open_January_2021_Europe/big-vs-hellraisers-m2-mirage.dem
16:00:51 [INFO] Setting demo id to big-vs-hellraisers-m2-mirage
16:00:51 [INFO] Setting parse rate to 128
16:00:51 [INFO] Go version>=1.14.0
16:00:51 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/Dreamhack_Open_January_202

16:00:54 [INFO] Setting parse rate to 128
16:00:54 [INFO] Go version>=1.14.0
16:00:54 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/IEM_KATOWICE_2021/furia-vs-natus-vincere-m1-mirage.dem
16:00:54 [INFO] Setting demo id to furia-vs-natus-vincere-m1-mirage
16:00:54 [INFO] Setting parse rate to 128
16:00:54 [INFO] Go version>=1.14.0
16:00:54 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/IEM_KATOWICE_2021/g2-vs-big-m2-mirage.dem
16:00:54 [INFO] Setting demo id to g2-vs-big-m2-mirage
16:00:54 [INFO] Setting parse rate to 128
16:00:54 [INFO] Go version>=1.14.0
16:00:54 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/IEM_KATOWICE_2021/g2-vs-spirit-m2-mirage.dem
16:00:54 [INFO] Setting demo id to g2-vs-spirit-m2-mirage
16:00:54 [INFO] Setting parse rate to 128
16:00:54 [INFO] Go version>=1.14.0
16:00:54 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Top_Players/IEM_KATOWICE_2021/gambit-vs-heroic-m3-mirage.dem
16:00:54 [INFO] 

# Demo Parsers: 71


In [26]:
mirageKills, mirageRunds, mirageDamages = generateData(mirage_parsers)

16:00:54 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:00:54 [INFO] Looking for file at /mnt/d/Top_Players/Blast_Premier_Global_Final_2020/complexity-vs-natus-vincere-m1-mirage.dem
16:01:30 [INFO] Wrote demo parse output to complexity-vs-natus-vincere-m1-mirage.json
16:01:30 [INFO] Reading in JSON from complexity-vs-natus-vincere-m1-mirage.json
16:01:30 [INFO] JSON data loaded, available in the `json` attribute to parser
16:01:30 [INFO] Successfully parsed JSON output
16:01:30 [INFO] Successfully returned JSON output
16:01:30 [INFO] Parsed kills to Pandas DataFrame
16:01:30 [INFO] Parsed rounds to Pandas DataFrame
16:01:30 [INFO] Parsed damages to Pandas DataFrame
16:01:30 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:01:30 [INFO] Looking for file at /mnt/d/Top_Players/Blast_Premier_Global_Final_2020/evil-geniuses-vs-furia-m2-mirage.dem
16:01:54 [INFO] Wrote demo parse output to evil-geniuses-vs-furia-m2-mirage.json
16:01:54

16:05:23 [INFO] Looking for file at /mnt/d/Top_Players/Blast_Premier_Spring_Showdown_2021/heroic-vs-9z-m1-mirage.dem
16:05:41 [INFO] Wrote demo parse output to heroic-vs-9z-m1-mirage.json
16:05:41 [INFO] Reading in JSON from heroic-vs-9z-m1-mirage.json
16:05:41 [INFO] JSON data loaded, available in the `json` attribute to parser
16:05:41 [INFO] Successfully parsed JSON output
16:05:41 [INFO] Successfully returned JSON output
16:05:41 [INFO] Parsed kills to Pandas DataFrame
16:05:41 [INFO] Parsed rounds to Pandas DataFrame
16:05:41 [INFO] Parsed damages to Pandas DataFrame
16:05:41 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:05:41 [INFO] Looking for file at /mnt/d/Top_Players/CS_SUMMIT_7/faze-vs-mibr-m3-mirage.dem
16:06:01 [INFO] Wrote demo parse output to faze-vs-mibr-m3-mirage.json
16:06:01 [INFO] Reading in JSON from faze-vs-mibr-m3-mirage.json
16:06:01 [INFO] JSON data loaded, available in the `json` attribute to parser
16:06:01 [INFO] Successfully p

16:10:08 [INFO] Successfully parsed JSON output
16:10:08 [INFO] Successfully returned JSON output
16:10:08 [INFO] Parsed kills to Pandas DataFrame
16:10:08 [INFO] Parsed rounds to Pandas DataFrame
16:10:08 [INFO] Parsed damages to Pandas DataFrame
16:10:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:10:08 [INFO] Looking for file at /mnt/d/Top_Players/Dreamhack Masters Spring 2021/g2-vs-mousesports-m2-mirage.dem
16:10:36 [INFO] Wrote demo parse output to g2-vs-mousesports-m2-mirage.json
16:10:36 [INFO] Reading in JSON from g2-vs-mousesports-m2-mirage.json
16:10:37 [INFO] JSON data loaded, available in the `json` attribute to parser
16:10:37 [INFO] Successfully parsed JSON output
16:10:37 [INFO] Successfully returned JSON output
16:10:37 [INFO] Parsed kills to Pandas DataFrame
16:10:37 [INFO] Parsed rounds to Pandas DataFrame
16:10:37 [INFO] Parsed damages to Pandas DataFrame
16:10:37 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parse

16:14:56 [INFO] Parsed kills to Pandas DataFrame
16:14:56 [INFO] Parsed rounds to Pandas DataFrame
16:14:56 [INFO] Parsed damages to Pandas DataFrame
16:14:56 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:14:56 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/complexity-vs-nip-m1-mirage.dem
16:15:25 [INFO] Wrote demo parse output to complexity-vs-nip-m1-mirage.json
16:15:25 [INFO] Reading in JSON from complexity-vs-nip-m1-mirage.json
16:15:25 [INFO] JSON data loaded, available in the `json` attribute to parser
16:15:25 [INFO] Successfully parsed JSON output
16:15:25 [INFO] Successfully returned JSON output
16:15:25 [INFO] Parsed kills to Pandas DataFrame
16:15:25 [INFO] Parsed rounds to Pandas DataFrame
16:15:25 [INFO] Parsed damages to Pandas DataFrame
16:15:25 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:15:25 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/ence-vs-g2-m2-mirage

16:19:30 [INFO] Wrote demo parse output to natus-vincere-vs-furia-m1-mirage.json
16:19:30 [INFO] Reading in JSON from natus-vincere-vs-furia-m1-mirage.json
16:19:30 [INFO] JSON data loaded, available in the `json` attribute to parser
16:19:30 [INFO] Successfully parsed JSON output
16:19:30 [INFO] Successfully returned JSON output
16:19:30 [INFO] Parsed kills to Pandas DataFrame
16:19:30 [INFO] Parsed rounds to Pandas DataFrame
16:19:30 [INFO] Parsed damages to Pandas DataFrame
16:19:30 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:19:30 [INFO] Looking for file at /mnt/d/Top_Players/ESL_Pro_League_Season_13/natus-vincere-vs-gambit-m2-mirage.dem
16:19:52 [INFO] Wrote demo parse output to natus-vincere-vs-gambit-m2-mirage.json
16:19:52 [INFO] Reading in JSON from natus-vincere-vs-gambit-m2-mirage.json
16:19:53 [INFO] JSON data loaded, available in the `json` attribute to parser
16:19:53 [INFO] Successfully parsed JSON output
16:19:53 [INFO] Successfully retu

16:24:17 [INFO] Successfully parsed JSON output
16:24:17 [INFO] Successfully returned JSON output
16:24:17 [INFO] Parsed kills to Pandas DataFrame
16:24:17 [INFO] Parsed rounds to Pandas DataFrame
16:24:17 [INFO] Parsed damages to Pandas DataFrame
16:24:17 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:24:17 [INFO] Looking for file at /mnt/d/Top_Players/IEM_KATOWICE_2021/g2-vs-spirit-m2-mirage.dem
16:24:41 [INFO] Wrote demo parse output to g2-vs-spirit-m2-mirage.json
16:24:41 [INFO] Reading in JSON from g2-vs-spirit-m2-mirage.json
16:24:41 [INFO] JSON data loaded, available in the `json` attribute to parser
16:24:41 [INFO] Successfully parsed JSON output
16:24:41 [INFO] Successfully returned JSON output
16:24:41 [INFO] Parsed kills to Pandas DataFrame
16:24:41 [INFO] Parsed rounds to Pandas DataFrame
16:24:41 [INFO] Parsed damages to Pandas DataFrame
16:24:41 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:24:41 [INFO] Looking 

In [27]:
for x in range(len(mirageKills)):
    if(mirageKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [217]:
mirageKill = concat_data(mirageKills)
mirageRound = concat_data(mirageRunds)
mirageDamage = concat_data(mirageDamages)

In [218]:
for x in range(len(mirageKill)):
    if  mirageKill.iloc[x]['AttackerAreaName'] != None:
        mirageKill.at[x, 'VictimAreaName'] = mappedPlaceName[mirageKill.iloc[x]['VictimAreaName']]
        mirageKill.at[x, 'AttackerAreaName'] = mappedPlaceName[mirageKill.iloc[x]['AttackerAreaName']]

for x in range(len(mirageDamage)):
    if  mirageDamage.iloc[x]['AttackerAreaName'] != None:
        mirageDamage.at[x, 'VictimAreaName'] = mappedPlaceName[mirageDamage.iloc[x]['VictimAreaName']]
        mirageDamage.at[x, 'AttackerAreaName'] = mappedPlaceName[mirageDamage.iloc[x]['AttackerAreaName']]

In [219]:
teams = {}
for x in mirageRound[mirageRound.MatchId == mirageRound.MatchId.unique()[0]].WinningTeam.unique():
    teams[x] = {'CT' : 0, 'T' : 0}
print(teams.keys())
for i, r in mirageRound[mirageRound.MatchId == mirageRound.MatchId.unique()[0]].iterrows():
    teams[r['WinningTeam']][r['WinningSide']] += 1
    if(r['WinningSide'] == 'CT'):
        teams[r['LosingTeam']]['T'] += 1
    else:
        teams[r['LosingTeam']]['CT'] += 1


dict_keys(['compLexity', 'Natus Vincere'])


In [220]:
teams

{'compLexity': {'CT': 18, 'T': 18}, 'Natus Vincere': {'CT': 18, 'T': 18}}

In [221]:
mirageRound.replace(to_replace=['Gambit Esports'], value=['Gambit'], inplace=True)
mirageKill.replace(to_replace=['Gambit Esports'], value=['Gambit'], inplace=True)

In [222]:
mirageRound.replace(to_replace=['NAVI GG.BET'], value=['Natus Vincere'], inplace=True)
mirageKill.replace(to_replace=['NAVI GG.BET'], value=['Natus Vincere'], inplace=True)

In [223]:
mirageRound.replace(to_replace=['NAVI'], value=['Natus Vincere'], inplace=True)
mirageKill.replace(to_replace=['NAVI'], value=['Natus Vincere'], inplace=True)

In [224]:
mirageRound.replace(to_replace=['TeamVitality'], value=['Team Vitality'], inplace=True)
mirageKill.replace(to_replace=['TeamVitality'], value=['Team Vitality'], inplace=True)

In [225]:
mirageRound.replace(to_replace=['BIG'], value=['BIG Clan'], inplace=True)
mirageKill.replace(to_replace=['BIG'], value=['BIG Clan'], inplace=True)

In [226]:
mirageRound.replace(to_replace=['FURIA eSports'], value=['FURIA'], inplace=True)
mirageKill.replace(to_replace=['FURIA eSports'], value=['FURIA'], inplace=True)

In [227]:
mirageRound.replace(to_replace=['compLexity'], value=['Complexity'], inplace=True)
mirageKill.replace(to_replace=['compLexity'], value=['Complexity'], inplace=True)

In [228]:
mirageRound.replace(to_replace=['Heroic GGBET'], value=['Heroic'], inplace=True)
mirageKill.replace(to_replace=['Heroic GGBET'], value=['Heroic'], inplace=True)

In [229]:
mirageRound.replace(to_replace=['HEROIC'], value=['Heroic'], inplace=True)
mirageKill.replace(to_replace=['HEROIC'], value=['Heroic'], inplace=True)

In [230]:
mirageRound.replace(to_replace=['FURIA Esports'], value=['FURIA'], inplace=True)
mirageKill.replace(to_replace=['FURIA Esports'], value=['FURIA'], inplace=True)

In [244]:
teams = {}
for x in mirageRound.WinningTeam.unique():
    teams[x] = {'CT' : 0, 'T' : 0}
print(teams.keys())
for i, r in mirageRound.iterrows():
    teams[r['WinningTeam']][r['WinningSide']] += 1
    if(r['WinningSide'] == 'CT'):
        teams[r['LosingTeam']]['T'] += 1
    else:
        teams[r['LosingTeam']]['CT'] += 1


dict_keys(['Complexity', 'Natus Vincere', 'FURIA', 'Evil Geniuses', 'G2 Esports', 'Team Vitality', 'MIBR', 'Ninjas in Pyjamas', 'BIG Clan', 'Team Spirit', 'Gambit', 'Heroic', '9z', 'FaZe Clan', 'mousesports', 'Fnatic', 'OG', 'Virtus.pro Parimatch', 'Fiend', 'paiN', 'HR Parimatch', 'forZe LUKOIL', 'FunPlus Phoenix', 'Sprout', 'ENCE', 'Team One', 'Team Liquid', 'Wisła All iN!', 'Renegades'])


In [245]:
for x in teams.keys():
    teams[x]['Players'] = []

In [249]:
for x in teams.keys():
    print(x)
    print(teams[x]['Players'])
    print("\n")

Complexity
['k0nfig', 'jks', 'JUGi', 'RUSH', 'blameF', 'poizon']


Natus Vincere
['electronic', 'Perfecto', 'flamie', 'Boombl4', 's1mple', 'b1t']


FURIA
['arT', 'VINI', 'yuurih', 'KSCERATO', 'honda', 'junior']


Evil Geniuses
['Ethan', 'Brehze', 'stanislaw', 'CeRq', 'tarik', 'oBo']


G2 Esports
['AmaNEk', 'NiKo', 'huNter-', 'nexa', 'kennyS', 'JACKZ']


Team Vitality
['ZywOo', 'RpK', 'shox ', 'apEX', 'misutaaa', 'shox', 'Kyojin']


MIBR
['shz', 'yel', 'danoco', 'boltz', 'chelo', 'exit', 'nak', 'yeL']


Ninjas in Pyjamas
['nawwk', 'REZ', 'Plopski', 'hampus', 'ztr', 'twist']


BIG Clan
['tabseN', 'XANTARES', 'k1to', 'syrsoN', 'tiziaN']


Team Spirit
['degster', 'mir', 'chopper', 'sdy', 'magixx', 'Patsi', 'Mir']


Gambit
['Hobbit', 'sh1ro', 'interz', 'nafany', 'Ax1Le', 'SH1R0', 'sh1r0']


Heroic
['TeSeS', 'refrezh', 'sjuush', 'cadiaN', 'stavn', 'niko', 'b0RUP', 'niko ']


9z
['try', 'BIT', 'EasTor', 'dgt', 'max']


FaZe Clan
['rain', 'olofmeister', 'coldzera', 'Kjaerbye', 'broky', 'Twistz

In [247]:
teams

{'Complexity': {'CT': 135, 'T': 161, 'Players': []},
 'Natus Vincere': {'CT': 166, 'T': 156, 'Players': []},
 'FURIA': {'CT': 213, 'T': 197, 'Players': []},
 'Evil Geniuses': {'CT': 45, 'T': 33, 'Players': []},
 'G2 Esports': {'CT': 141, 'T': 176, 'Players': []},
 'Team Vitality': {'CT': 90, 'T': 70, 'Players': []},
 'MIBR': {'CT': 88, 'T': 90, 'Players': []},
 'Ninjas in Pyjamas': {'CT': 67, 'T': 76, 'Players': []},
 'BIG Clan': {'CT': 73, 'T': 88, 'Players': []},
 'Team Spirit': {'CT': 101, 'T': 87, 'Players': []},
 'Gambit': {'CT': 69, 'T': 120, 'Players': []},
 'Heroic': {'CT': 116, 'T': 85, 'Players': []},
 '9z': {'CT': 9, 'T': 15, 'Players': []},
 'FaZe Clan': {'CT': 77, 'T': 95, 'Players': []},
 'mousesports': {'CT': 102, 'T': 84, 'Players': []},
 'Fnatic': {'CT': 57, 'T': 55, 'Players': []},
 'OG': {'CT': 28, 'T': 45, 'Players': []},
 'Virtus.pro Parimatch': {'CT': 60, 'T': 47, 'Players': []},
 'Fiend': {'CT': 10, 'T': 15, 'Players': []},
 'paiN': {'CT': 15, 'T': 13, 'Players':

In [248]:
for x in mirageKill.AttackerName.unique():
    print(x)
    if x != None and x not in teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players'] :
        teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players'].append(x)

k0nfig
jks
electronic
JUGi
Perfecto
RUSH
flamie
Boombl4
blameF
s1mple
None
Ethan
arT
VINI
Brehze
yuurih
KSCERATO
honda
stanislaw
CeRq
tarik
AmaNEk
NiKo
huNter-
nexa
kennyS
ZywOo
RpK
shox 
apEX
misutaaa
poizon
shz
yel
danoco
boltz
chelo
nawwk
tabseN
REZ
XANTARES
Plopski
k1to
hampus
syrsoN
ztr
tiziaN
junior
exit
degster
JACKZ
mir
chopper
sdy
magixx
Hobbit
sh1ro
interz
nafany
Ax1Le
TeSeS
try
BIT
EasTor
refrezh
sjuush
cadiaN
dgt
stavn
max
nak
rain
olofmeister
coldzera
Kjaerbye
broky
ropz
Bymas
acoR
karrigan
frozen
Jackinho
KRIMZ
Brollan
Golden
JW
niko
b0RUP
mantuu
NBK-
Aleksib
ISSAA
valde
twist
YEKINDAR
Qikert
Jame
buster
SANJI
v1c7oR
dream3r
REDSTAR
bubble
mar
b1t
PKL
biguzera
NEKiZ
saffee
hardzao
dexter
SH1R0
Twistzz
shox
Kyojin
JIaYm
ProbLeM
anarkez
kalinka
Flarich
FL1T
Patsi
Mir
almazer
Jerry
Aunkere
Woro2k
farlig
chrisJ
maden
STYKO
zehN
denis
slaxz-
faveN
kressy
Spiidi
Snappi
dycha
doto
allu
Spinx
oBo
sh1r0
prt
malbsMd
pesadelo
Skullz
Maluk3
yeL
qikert
Grim
FalleN
NAF
EliGE
Stewie2K
n

In [236]:
mirageKill.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'AssisterSteamId', 'AssisterName', 'AssisterTeam',
       'AssisterSide', 'AssisterX', 'AssisterY', 'AssisterZ', 'AssisterAreaId',
       'AssisterAreaName', 'IsWallbang', 'IsFlashed', 'IsHeadshot', 'IsTrade',
       'PlayerTradedName', 'PlayerTradedTeam', 'PlayerTradedSteamId',
       'IsFirstKill', 'Weapon', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [237]:
player_names = list(mirageKill.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [238]:
player_name

{'s1mple': ['s1mple'],
 'KSCERATO': ['KSCERATO'],
 'arT': ['arT'],
 'VINI': ['VINI'],
 'yuurih': ['yuurih'],
 'huNter-': ['huNter-'],
 'electronic': ['electronic'],
 'junior': ['junior'],
 'jks': ['jks'],
 'nexa': ['nexa'],
 'Boombl4': ['Boombl4'],
 'AmaNEk': ['AmaNEk'],
 'NiKo': ['NiKo'],
 'Perfecto': ['Perfecto'],
 'blameF': ['blameF'],
 'k0nfig': ['k0nfig'],
 'chelo': ['chelo'],
 'stavn': ['stavn'],
 'frozen': ['frozen'],
 'Ax1Le': ['Ax1Le'],
 'degster': ['degster'],
 'Hobbit': ['Hobbit'],
 'poizon': ['poizon'],
 'b1t': ['b1t'],
 'ropz': ['ropz'],
 'ZywOo': ['ZywOo'],
 'chopper': ['chopper'],
 'tabseN': ['tabseN'],
 'cadiaN': ['cadiaN'],
 'TeSeS': ['TeSeS'],
 'RUSH': ['RUSH'],
 'broky': ['broky'],
 'Bymas': ['Bymas'],
 'nafany': ['nafany'],
 'hampus': ['hampus'],
 'misutaaa': ['misutaaa'],
 'JACKZ': ['JACKZ'],
 'shz': ['shz'],
 'refrezh': ['refrezh'],
 'acoR': ['acoR'],
 'coldzera': ['coldzera'],
 'XANTARES': ['XANTARES'],
 'boltz': ['boltz'],
 'sdy': ['sdy'],
 'magixx': ['magixx'],

In [239]:
del player_name['k0nfig ']
player_name['k0nfig'].append('k0nfig ')
mirageDamage.replace(to_replace=['k0nfig '], value=['k0nfig'], inplace=True)
mirageKill.replace(to_replace=['k0nfig '], value=['k0nfig'], inplace=True)

In [240]:
mirageDamage.replace(to_replace=['k0nfig '], value=['k0nfig'], inplace=True)
mirageKill.replace(to_replace=['k0nfig '], value=['k0nfig'], inplace=True)

In [241]:
del player_name['blameF ']
player_name['blameF'].append('blameF ')
mirageDamage.replace(to_replace=['blameF '], value=['blameF'], inplace=True)
mirageKill.replace(to_replace=['blameF '], value=['blameF'], inplace=True)

In [242]:
del player_name['RUSH ']
player_name['RUSH'].append('RUSH ')
mirageDamage.replace(to_replace=['RUSH '], value=['RUSH'], inplace=True)
mirageKill.replace(to_replace=['RUSH '], value=['RUSH'], inplace=True)

In [243]:
del player_name['NiKo ']
player_name['NiKo'].append('NiKo ')
mirageDamage.replace(to_replace=['NiKo '], value=['NiKo'], inplace=True)
mirageKill.replace(to_replace=['NiKo '], value=['NiKo'], inplace=True)

In [250]:
del player_name['yeL']
player_name['yel'].append('yeL')
mirageDamage.replace(to_replace=['yeL'], value=['yel'], inplace=True)
mirageKill.replace(to_replace=['yeL'], value=['yel'], inplace=True)

In [251]:
del player_name['Mir']
player_name['mir'].append('Mir')
mirageDamage.replace(to_replace=['Mir'], value=['mir'], inplace=True)
mirageKill.replace(to_replace=['Mir'], value=['mir'], inplace=True)

In [252]:
del player_name['SH1R0']
player_name['sh1ro'].append('SH1R0')
mirageDamage.replace(to_replace=['SH1R0'], value=['sh1ro'], inplace=True)
mirageKill.replace(to_replace=['SH1R0'], value=['sh1ro'], inplace=True)

In [253]:
del player_name['sh1r0']
player_name['sh1ro'].append('sh1r0')
mirageDamage.replace(to_replace=['sh1r0'], value=['sh1ro'], inplace=True)
mirageKill.replace(to_replace=['sh1r0'], value=['sh1ro'], inplace=True)

In [254]:
del player_name['niko ']
player_name['niko'].append('niko ')
mirageDamage.replace(to_replace=['niko '], value=['niko'], inplace=True)
mirageKill.replace(to_replace=['niko '], value=['niko'], inplace=True)

In [255]:
del player_name['qikert']
player_name['Qikert'].append('qikert')
mirageDamage.replace(to_replace=['qikert'], value=['Qikert'], inplace=True)
mirageKill.replace(to_replace=['qikert'], value=['Qikert'], inplace=True)

In [264]:
def createSidedCSVDFWithScaledPer30(players, damageData, killData, roundData, teams):
    locationNames = []
    playerNames = []

    Kills = []
    killsPer30 = []
    Deaths = []
    deathsPer30 = []
    KdDifferential = []
    kdPer30 = []
    DamageDealt = []
    damageDealtPer30 = []
    DamageTaken = []
    damageTakenPer30 = []
    DamageDifferential = []
    damageDiffPer30 = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            for y in teams.keys():
                if x in teams[y]['Players']:
                    totalRounds.append(teams[y]['CT'])
                    totalRounds.append(teams[y]['T'])
                    break
            
            damageDiffPer30.append(tempCTDamageDifferential / totalRounds[-2] * 30)
            damageDiffPer30.append(tempTDamageDifferential / totalRounds[-1] * 30)

            kdPer30.append(tempCTKdDifferential / totalRounds[-2] * 30)
            kdPer30.append(tempTKdDifferential / totalRounds[-1] * 30)
            
            
            killsPer30.append(tempCTKills / totalRounds[-2] * 30)
            killsPer30.append(tempTKills / totalRounds[-1] * 30)
            
            deathsPer30.append(tempCTDeaths / totalRounds[-2] * 30)
            deathsPer30.append(tempTDeaths / totalRounds[-1] * 30)

            damageDealtPer30.append(tempCTDamageDealt / totalRounds[-2] * 30)
            damageDealtPer30.append(tempTDamageDealt / totalRounds[-1] * 30)
            
            damageTakenPer30.append(tempCTDamageTaken / totalRounds[-2] * 30)
            damageTakenPer30.append(tempTDamageTaken / totalRounds[-1] * 30)
            
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    dataframe['Damage Diff Per 30'] = damageDiffPer30
    dataframe['KD Diff Per 30'] = kdPer30
    dataframe['Kills Per 30'] = killsPer30
    dataframe['Deaths Per 30'] = deathsPer30
    dataframe['Damage Dealt Per 30'] = damageDealtPer30
    dataframe['Damage Taken Per 30'] = damageTakenPer30   
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [265]:
damageDifferentials = createSidedCSVDFWithScaledPer30(player_name, mirageDamage, mirageKill, mirageRound, teams)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Boolean Serie

In [266]:
damageDifferentials

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds,Damage Diff Per 30,KD Diff Per 30,Kills Per 30,Deaths Per 30,Damage Dealt Per 30,Damage Taken Per 30
0,Ticketbooth,s1mple,22,10,12,1814,1248,566,CT,166,102.289157,2.168675,3.975904,1.807229,327.831325,225.542169
1,Ticketbooth,s1mple,2,1,1,200,122,78,T,156,15.000000,0.192308,0.384615,0.192308,38.461538,23.461538
2,Default A,s1mple,8,2,6,761,205,556,CT,166,100.481928,1.084337,1.445783,0.361446,137.530120,37.048193
3,Default A,s1mple,6,5,1,592,400,192,T,156,36.923077,0.192308,1.153846,0.961538,113.846154,76.923077
4,CT Side Apps,s1mple,0,0,0,0,0,0,CT,166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16301,A Site,Flarich,0,1,-1,0,100,-100,T,8,-375.000000,-3.750000,0.000000,3.750000,0.000000,375.000000
16302,Default A,Flarich,0,0,0,0,0,0,CT,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16303,Default A,Flarich,0,1,-1,0,100,-100,T,8,-375.000000,-3.750000,0.000000,3.750000,0.000000,375.000000
16304,CT Spawn,nak,1,1,0,0,0,0,CT,88,0.000000,0.000000,0.340909,0.340909,0.000000,0.000000


In [267]:
damageDifferentials.to_csv(base_output_path + "damageDifferentialsScaled.csv")

In [30]:
mirageKill.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'AssisterSteamId', 'AssisterName', 'AssisterTeam',
       'AssisterSide', 'AssisterX', 'AssisterY', 'AssisterZ', 'AssisterAreaId',
       'AssisterAreaName', 'IsWallbang', 'IsFlashed', 'IsHeadshot', 'IsTrade',
       'PlayerTradedName', 'PlayerTradedTeam', 'PlayerTradedSteamId',
       'IsFirstKill', 'Weapon', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [31]:
mirageDamage.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimSteamId',
       'VictimName', 'VictimTeam', 'VictimSide', 'VictimX', 'VictimY',
       'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX',
       'VictimViewY', 'Weapon', 'HpDamage', 'HpDamageTaken', 'ArmorDamage',
       'ArmorDamageTaken', 'HitGroup', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [32]:
mirageAwpKills = mirageKill[mirageKill['Weapon'] == 'AWP']

In [33]:
mirageAwpDamage = mirageDamage[mirageDamage['Weapon'] == 'AWP']

In [34]:
player_names = list(mirageAwpKills.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [38]:
player_name['sh1r0'].append('SH1R0')

In [39]:
del player_name['SH1R0']

In [57]:
ctplayer_names = list(mirageCTAWP.AttackerName.value_counts().index)
ctplayer_name = {}
for x in ctplayer_names:
    ctplayer_name[x] = [x]

In [58]:
tplayer_names = list(mirageTAWP.AttackerName.value_counts().index)
tplayer_name = {}
for x in tplayer_names:
    tplayer_name[x] = [x]

In [63]:
ctplayer_name['sh1r0'].append('sh1ro')

In [64]:
del ctplayer_name['sh1ro']

In [65]:
ctplayer_name

{'junior': ['junior'],
 's1mple': ['s1mple'],
 'arT': ['arT'],
 'ZywOo': ['ZywOo'],
 'cadiaN': ['cadiaN'],
 'degster': ['degster'],
 'poizon': ['poizon'],
 'acoR': ['acoR'],
 'stavn': ['stavn'],
 'AmaNEk': ['AmaNEk'],
 'Jame': ['Jame'],
 'broky': ['broky'],
 'k0nfig': ['k0nfig'],
 'yel': ['yel'],
 'nawwk': ['nawwk'],
 'syrsoN': ['syrsoN'],
 'sh1r0': ['sh1r0', 'SH1R0', 'sh1ro'],
 'Maluk3': ['Maluk3'],
 'FalleN': ['FalleN'],
 'ropz': ['ropz'],
 'kennyS': ['kennyS'],
 'shox': ['shox'],
 'CeRq': ['CeRq'],
 'yeL': ['yeL'],
 'Woro2k': ['Woro2k'],
 'Sico': ['Sico'],
 'NiKo': ['NiKo'],
 'hades': ['hades'],
 'allu': ['allu'],
 'anarkez': ['anarkez'],
 'buster': ['buster'],
 'JW': ['JW'],
 'farlig': ['farlig'],
 'JUGi': ['JUGi'],
 'k0nfig ': ['k0nfig '],
 'k1to': ['k1to'],
 'mantuu': ['mantuu'],
 'almazer': ['almazer'],
 'shox ': ['shox '],
 'Stewie2K': ['Stewie2K'],
 'slaxz-': ['slaxz-'],
 'JACKZ': ['JACKZ'],
 'SZPERO': ['SZPERO'],
 'saffee': ['saffee'],
 'magixx': ['magixx'],
 'Snappi': ['Snap

In [78]:
tplayer_name['sh1r0'].append('SH1R0')

In [79]:
del tplayer_name['SH1R0']

In [42]:
mirageCTAWP = mirageAwpKills[mirageAwpKills['AttackerSide'] == 'CT']
mirageTAWP = mirageAwpKills[mirageAwpKills['AttackerSide'] == 'T']

In [43]:
mirageCTAWPdamage = mirageAwpDamage[mirageAwpDamage['AttackerSide'] == 'CT']
mirageTAWPdamage = mirageAwpDamage[mirageAwpDamage['AttackerSide'] == 'T']

In [48]:
mirageCTAWP

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
39,98602,92.937008,7.656120e+16,s1mple,Natus Vincere,CT,-462.487610,-2300.874268,-169.156998,237.0,...,False,False,RUSH,compLexity,7.656120e+16,False,AWP,7,complexity-vs-natus-vincere-m1-mirage,de_mirage
57,148526,94.181102,7.656120e+16,s1mple,Natus Vincere,CT,-1845.204712,5.891641,-167.968750,2536.0,...,False,False,JUGi,compLexity,7.656120e+16,False,AWP,10,complexity-vs-natus-vincere-m1-mirage,de_mirage
67,179459,111.314961,7.656120e+16,s1mple,Natus Vincere,CT,-713.930481,-963.229980,-215.968750,225.0,...,False,False,Boombl4,Natus Vincere,7.656120e+16,False,AWP,11,complexity-vs-natus-vincere-m1-mirage,de_mirage
91,238736,9.488189,7.656120e+16,flamie,Natus Vincere,CT,-923.293457,-403.907410,-48.093750,360.0,...,True,False,None,None,NaN,True,AWP,15,complexity-vs-natus-vincere-m1-mirage,de_mirage
123,326869,8.173228,7.656120e+16,JUGi,compLexity,CT,-1186.206299,-725.536987,-128.968750,1074.0,...,False,False,None,None,NaN,True,AWP,19,complexity-vs-natus-vincere-m1-mirage,de_mirage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12420,195449,85.622047,7.656120e+16,Sico,Renegades,CT,-1011.622559,-2416.895508,-167.968750,56.0,...,False,False,malta,Renegades,7.656120e+16,False,AWP,13,teamone-vs-renegades-m2-mirage,de_mirage
12421,196043,90.299213,7.656120e+16,Sico,Renegades,CT,-1335.075317,-2364.855469,-209.864578,1175.0,...,False,False,pesadelo,Team One,7.656120e+16,False,AWP,13,teamone-vs-renegades-m2-mirage,de_mirage
12429,208828,64.401575,7.656120e+16,Sico,Renegades,CT,-1939.790771,-641.188538,-167.968750,9.0,...,False,False,Hatz,Renegades,7.656120e+16,False,AWP,14,teamone-vs-renegades-m2-mirage,de_mirage
12432,217422,15.803150,7.656120e+16,Sico,Renegades,CT,-1181.989380,-739.492188,-167.968750,1074.0,...,False,False,None,None,NaN,True,AWP,15,teamone-vs-renegades-m2-mirage,de_mirage


In [82]:
mirageCTAWPDifferentials = createDifferentialDF(ctplayer_name, mirageCTAWPdamage, mirageCTAWP, mirageRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [83]:
mirageTAWPDifferentials = createDifferentialDF(player_name, mirageTAWPdamage, mirageTAWP, mirageRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [86]:
mirageTAWPDifferentials['Percentage Kills'] = mirageTAWPDifferentials['Kills'] / mirageTAWPDifferentials['Kills'].sum() * 100
mirageTAWPDifferentials['Percentage Deaths'] = mirageTAWPDifferentials['Deaths'] / mirageTAWPDifferentials['Deaths'].sum() * 100
mirageTAWPDifferentials['Percentage Damage Taken'] = mirageTAWPDifferentials['Damage Taken'] / mirageTAWPDifferentials['Damage Taken'].sum() * 100
mirageTAWPDifferentials['Percentage Damage Dealt'] = mirageTAWPDifferentials['Damage Dealt'] / mirageTAWPDifferentials['Damage Dealt'].sum() * 100

In [87]:
mirageCTAWPDifferentials['Percentage Kills'] = mirageCTAWPDifferentials['Kills'] / mirageCTAWPDifferentials['Kills'].sum() * 100
mirageCTAWPDifferentials['Percentage Deaths'] = mirageCTAWPDifferentials['Deaths'] / mirageCTAWPDifferentials['Deaths'].sum() * 100
mirageCTAWPDifferentials['Percentage Damage Taken'] = mirageCTAWPDifferentials['Damage Taken'] / mirageCTAWPDifferentials['Damage Taken'].sum() * 100
mirageCTAWPDifferentials['Percentage Damage Dealt'] = mirageCTAWPDifferentials['Damage Dealt'] / mirageCTAWPDifferentials['Damage Dealt'].sum() * 100

In [88]:
mirageCTAWPDifferentials.to_csv(base_output_path + "mirageCTAwpDifferentials.csv")

In [89]:
mirageTAWPDifferentials.to_csv(base_output_path + "mirageTAwpDifferentials.csv")

In [40]:
player_name

{'s1mple': ['s1mple'],
 'junior': ['junior'],
 'arT': ['arT'],
 'degster': ['degster'],
 'cadiaN': ['cadiaN'],
 'poizon': ['poizon'],
 'acoR': ['acoR'],
 'ZywOo': ['ZywOo'],
 'Jame': ['Jame'],
 'broky': ['broky'],
 'syrsoN': ['syrsoN'],
 'nawwk': ['nawwk'],
 'sh1r0': ['sh1r0', 'sh1ro', 'SH1R0'],
 'stavn': ['stavn'],
 'AmaNEk': ['AmaNEk'],
 'NiKo': ['NiKo'],
 'yel': ['yel'],
 'kennyS': ['kennyS'],
 'Maluk3': ['Maluk3'],
 'JW': ['JW'],
 'yeL': ['yeL'],
 'k0nfig': ['k0nfig'],
 'JUGi': ['JUGi'],
 'hades': ['hades'],
 'mantuu': ['mantuu'],
 'FalleN': ['FalleN'],
 'Woro2k': ['Woro2k'],
 'ropz': ['ropz'],
 'shox': ['shox'],
 'CeRq': ['CeRq'],
 'anarkez': ['anarkez'],
 'Sico': ['Sico'],
 'allu': ['allu'],
 'buster': ['buster'],
 'farlig': ['farlig'],
 'KSCERATO': ['KSCERATO'],
 'k0nfig ': ['k0nfig '],
 'magixx': ['magixx'],
 'k1to': ['k1to'],
 'shox ': ['shox '],
 'slaxz-': ['slaxz-'],
 'saffee': ['saffee'],
 'almazer': ['almazer'],
 'Stewie2K': ['Stewie2K'],
 'JACKZ': ['JACKZ'],
 'try': ['try

In [85]:
mirageAWPDifferentials = createSidedCSVDF(player_name, mirageAwpDamage, mirageAwpKills, mirageRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [100]:
mirageAWPDifferentials.columns

Index(['Location', 'Player', 'Kills', 'Deaths', 'K/D', 'Damage Dealt',
       'Damage Taken', 'Damage Differential', 'Side', 'Total Rounds',
       'Percentage Damage Dealt'],
      dtype='object')

In [85]:
mirageAWPDifferentials['Percentage Kills'] = mirageAWPDifferentials['Kills'] / mirageAWPDifferentials['Kills'].sum() * 100
mirageAWPDifferentials['Percentage Deaths'] = mirageAWPDifferentials['Deaths'] / mirageAWPDifferentials['Deaths'].sum() * 100
mirageAWPDifferentials['Percentage Damage Taken'] = mirageAWPDifferentials['Damage Taken'] / mirageAWPDifferentials['Damage Taken'].sum() * 100
mirageAWPDifferentials['Percentage Damage Dealt'] = mirageAWPDifferentials['Damage Dealt'] / mirageAWPDifferentials['Damage Dealt'].sum() * 100

NameError: name 'mirageAWPDifferentials' is not defined

In [104]:
mirageAWPDifferentials.to_csv(base_output_path + "mirageAwpDifferentials.csv")

In [1]:
mirageAWPDifferentials

NameError: name 'mirageAWPDifferentials' is not defined